In [13]:
from ipywidgets import FileUpload, Button, Output, HBox, VBox, Layout
from ipywidgets import HTML as htmlWidget
from IPython.display import display, clear_output, Image, HTML
import subprocess, os, base64
import markdown


output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')
loadingGifPath = 'https://upload.wikimedia.org/wikipedia/commons/a/a3/Lightness_rotate_36f_cw.gif'

def convert(b):
    with output:
        clear_output()
        display(Image(url=loadingGifPath, width=25, height=25))
        try:
            nbName = list(fileInput.value.keys())[0]
        except IndexError:
            clear_output()
            print()
            display(HTML('<p style="color:red">Have you uploaded a notebook file yet? </p>'))
            return 
        pdfName = nbName.replace('ipynb', 'pdf')
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0])
        proc = subprocess.run(['jupyter', 'nbconvert', nbName, '--to', 'pdf'])
        clear_output()
        if proc.returncode == 1:
            print()
            display(HTML('<p style="color:red">LaTeX compiling error! </p>'))
        else:
            with open(pdfName, "rb") as f:
                data = f.read()
                b64 = base64.b64encode(data)
                payload = b64.decode()
            display(HTML(f'''<a download="{pdfName}" href="data:text/csv;base64,{payload}" target="_blank">  
                            <button class="p-Widget jupyter-widgets jupyter-button widget-button"> Download PDF </button>
                        </a>'''))

convertButton.on_click(convert)

# display(fileInput, convertButton, output) 


text = '''
## 3 Easy Steps: 
1. Upload your notebook (.ipynb) file. You may only convert one notebook at a time \n
* Hit Convert \n
* A Download button will show up when your PDF file is ready. Enjoy!
'''

footer = '''<br><br/><p>Powered by&emsp;<p/>
<img src="https://raw.githubusercontent.com/beginnerSC/misc/master/docs/source/ds/poweredby.png" alt="Binder, Jupyter, Voilà" width="250"/>
'''

box_layout = Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')
HBox([
    VBox([
        htmlWidget(markdown.markdown('# Notebook to PDF Converter')), 
        HBox([htmlWidget(markdown.markdown(text)), htmlWidget('<p>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<p/>'), 
            VBox([htmlWidget('<br><br/>'), fileInput, convertButton, output])]), 
        htmlWidget(markdown.markdown(footer))
    ])
], layout=box_layout)
